# **Advanced Techniques in Data Analysis - MACHINE LEARNING PROJECT** 

## Objectives and Motivation

In this project we want to make a deep dive into the world of stock prediction, more specifically on the PayPal stock. Our objectives are:

- Make a rigorous time series analysis 

- Make a classifier to evaluate if the stock will go up or down

- Risk management

# Importing the Paypal Stock information

## Imports and Data Cleaning

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import sklearn.svm as svc
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

Now we download the Standard & Poor 500 dataset (or keep it updated)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yash16jr/s-and-p500-daily-update-dataset")

print("Path to dataset files:", path)

In [ ]:

import os

path = "/Users/guilhermealves/.cache/kagglehub/datasets/yash16jr/s-and-p500-daily-update-dataset/versions/275"

print(os.listdir(path))

In [ ]:

df = pd.read_csv(
    
    '/Users/guilhermealves/.cache/kagglehub/datasets/yash16jr/s-and-p500-daily-update-dataset/versions/275/SnP_daily_update.csv',
    
    header=[0, 1],         
    index_col=0,          
    parse_dates=True      
)

print(df.head())


In [ ]:
df_pypl_completo = df.xs('PYPL', level=1, axis=1)

In [ ]:
df_pypl_completo.columns = [col.lower() for col in df_pypl_completo.columns]

if 'price' in df_pypl_completo.columns:
    df_pypl_completo = df_pypl_completo.drop(columns=['price'])

df_pypl_completo.dropna(subset=['close'], inplace=True)

df_pypl = df_pypl_completo.copy()

print("PayPal's DataFrame (PYPL) Clean and Ready:")
print(df_pypl.head())
print(f"Column number: {df_pypl.shape[1]}")
print(f"Existent Columns: {list(df_pypl.columns)}")


# Time Series Analysis


Stock market predicition must be preceded by a rigorous Time Series Analysis. Financial data is unique because observations are not independent and identifcally distributed, they exibit autocorrelation and non-stationarity.


Time Series Analysis serves two critical functions:

- Risk Management: Quantifying extreme events. This will provide context for interpreting machine learning models.

- Validation and feature engineering: It allow us to confirm the statistical properties of data (Stationarity and other)

## Discrete Analysis

### Logarithmic Return 

### Histogram of returns

In [ ]:
ret = 100 * (df_pypl['close'].pct_change())
plt.figure(figsize=(10,6))
plt.hist(ret, bins=50, density = True, label = "Daily Returns (PayPal)", color = "skyblue", alpha=0.7)

from scipy.stats import norm

mu, sigma = ret.mean(), ret.std()

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, sigma) 


plt.plot(x, p, 'k', linewidth=2, label='Normal Curve')


title = "Histograma de Retornos Diários da PayPal vs. Distribuição Normal"
plt.title(title, fontsize=15)
plt.xlabel("Retornos Diários (%)", fontsize=12)
plt.ylabel("Frequência Normalizada", fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
from scipy.stats import jarque_bera, skew, kurtosis
import pandas as pd


ret_clean = ret.dropna() 


jb_result = jarque_bera(ret_clean)
jb_statistic = jb_result[0]
jb_pvalue = jb_result[1]


skewness_value = skew(ret_clean)

kurtosis_value = kurtosis(ret_clean, fisher=False) 

print(f"Estatística Jarque-Bera: {jb_statistic:.4f}")
print(f"P-value: {jb_pvalue:.4f}")
print(f"Assimetria (Skewness): {skewness_value:.4f}")
print(f"Curtose (Kurtosis): {kurtosis_value:.4f}")

# Regra de Decisão
if jb_pvalue <= 0.05:
    print("Conclusão: Rejeitamos a Hipótese Nula (H0). Os retornos NÃO são normalmente distribuídos (devido a caudas gordas ou assimetria).")
else:
    print("Conclusão: Não rejeitamos H0.")

## Stationary Test

## Time Dependency Analysis

# PayPal Stock Prediction using a Support Vector Machine 


The idea of Support Vector Machine is by finding a hyperplane to divide the data into groups, this will classify if the stock is going up or down based on historic data.

This only shows history up to 2015-07-06 because PayPal only turned public in July of 2015 (The S&P500 file starts at 2010)

Since the Support Vector Machine is a classifier we need to define the target

- 1 if tomorrow's price is BIGGER than today
- 0 if tomorrow's price is SMALLER than today

In [ ]:
df_pypl["Target"]= np.where(df_pypl["close"].shift(-1) > df_pypl["close"], 1, 0)

In [ ]:
df_pypl["SMA_20"]= df_pypl["close"].rolling(window=20).mean()

In [ ]:
df_pypl["SMA_50"]=df_pypl["close"].rolling(window=50).mean()

In [ ]:
df_ml_final = df_pypl.dropna()

In [ ]:
features = ["open", "high", "low", "close", "volume", "SMA_20", "SMA_50"]
X = df_ml_final [features]
y = df_ml_final["Target"]

In [ ]:
print( "DataFrame da Paypal com features e Target")
print(df_ml_final[features + ["Target"]].tail())
print(f"\nNúmero de amostras prontas para o SVM (X e y): {X.shape[0]}")
print(f"Número de Features (X): {X.shape[1]}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=False)

In [ ]:
np.random.seed(42)
num_samples= X.shape[0]

scaler = StandardScaler()
X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
model = SVC (kernel= "rbf", C=1.0, gamma = "scale", random_state=42)

In [ ]:
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
cm= confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['DOWN (0)', 'UP (1)']))

# Risk Management on PayPal Stocks

Risk Management is fundamental to predict the behavior of stocks. Markowitz stated in his famous book, that volatility, which was a phenomenon not closely linked to math, to be the standard deviation of returns, $\sigma_r$.

There are Several methods to predict volatility, we will start with 2 classical ones and then move to a machine learning approach, with support vector regressors and neural networks. 

But first we will calculate the return volatility by using the formula for the Realized Volatility:

$$ \sigma=\sqrt{\frac{1}{n-1}\sum_{n=1}^N (r_n -\mu_r)^2} $$

where $N$ is the number of observations, $r_n$ is the return at observation $n$ and $\mu_r$ is the average return

In [ ]:
ret = 100 * (df_pypl['close'].pct_change()).dropna()
realized_vol = ret.rolling(5).std()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(realized_vol, label='Volatilidade Realizada (5 dias)', color='blue')
plt.title('Volatilidade Realizada da PayPal (PYPL)')
plt.xlabel('Data')
plt.ylabel('Volatilidade (%)')
plt.legend()
plt.show()

This is interesting because we can see big spikes on 2020 and 2022:

- 2020: Begining of the COVID pandemic

- 2022: A shocking report about the Q4 2021 was released. The sharp drop was due to the company drastically cutting its growth forecasts.

In [ ]:
retv = ret.values

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_pypl.index[1:], retv, label='Retornos Diários (%)', color='green')
plt.title('Volatility clustering of PayPal (PYPL)')
plt.xlabel('Date')
plt.ylabel('Daily Returns (%)')
plt.legend()
plt.show()

with this volatility clustering we can now in which direction was the spikes on 2020 and 2022 (downwards)

## ARCH Model

The ARCH model was one of the first statistical models introduced to predict volatility: the ARCH model is a univariate model and based on historical asset returns

$$\sigma_t ^2 = \omega + \sum_{k=1}^p \alpha_k (r_{t - k})^2$$

where the mean model is:

$$ r_t = \sigma_t \epsilon_t $$

where $\epsilon_t$ is assumed to be normally distributed

In this project we will not implement the ARCH model from the ground, instead we will use the arch library

In [ ]:
from arch import arch_model
from sklearn.metrics import mean_squared_error as mse

In [ ]:
arch = arch_model(ret, mean="Zero", vol="ARCH", p=1).fit(disp="off")
print(arch.summary())

In [ ]:
n=252
split_date=ret.iloc[-n:].index


In [ ]:
bic_arch = []
for p in range(1, 5):
    arch = arch_model(ret, mean='zero', vol='ARCH', p=p).fit(disp='off')
    bic_arch.append(arch.bic)
    if arch.bic == np.min(bic_arch):
        best_param = p
arch = arch_model(ret, mean='zero', vol='ARCH', p=best_param).fit(disp='off')
print(arch.summary())

forecast_arch = arch.forecast(start=split_date[0])


In [ ]:
rmse_arch=np.sqrt(mse(realized_vol[-n:] /100 , np.sqrt(forecast_arch.variance.iloc[-len(split_date): ] /100 )))
print(f"RMSE for ARCH model: {rmse_arch}")

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(realized_vol /100 , label= "Realized Volatility")
plt.plot(np.sqrt(forecast_arch.variance.iloc[-len(split_date):]) /100 , label = "Volatility Prediction - ARCH")
plt.title("Volatility Prediction with ARCH", fontsize = 12)
plt.legend()
plt.show()

As we can see we got a nice prediction of the volatility, even using an old model such as ARCH.

Disadvantages of using ARCH:
- Needs a lot of parameters: Markets are complex, and ARCH model can not capture all the shocks of volatility in data using a small $p$. To get a good modelation, a higher $p$ would be necessary

- Non-negativity: Two of the assumptions of ARCH are that $\alpha_k$ and $\omega$ are $>0$ which turns the volatility hard to model.

- Information assimetry: ARCH only looks for past returns, $r_{t-k} ^2$ to predict future volatility, $\sigma_t ^2$, but ignores the direction of the shock. This means that the model cannot capture the leverage effect: Bad news tend to a higher volatility and good news tends to lower volatility.

## GARCH model

GARCH is an extension of ARCH incorporating lagged conditional variance. This makes the model multivariate in the sense that it is an autoregressive moving average model.

In [ ]:
bic_garc= []

for p in range (1,5):
    for q in range(1,5):
        garch = arch_model(ret, mean="zero", vol="GARCH", p=p, o=0, q=q).fit(disp="off")
        bic_garc.append(garch.bic)
        if garch.bic == np.min(bic_garc):
            best_param = (p,q)
garch = arch_model(ret, mean="zero", vol="GARCH", p=best_param[0], o=0, q=best_param[1]).fit(disp="off")
print(garch.summary())
forecast_garch = garch.forecast(start=split_date[0])
forecast_garch 

In [ ]:
rmse_garch = np.sqrt(mse(realized_vol[-n:] /100 , np.sqrt(forecast_garch.variance.iloc[-len(split_date): ] /100 )))
print(f"RMSE for GARCH model: {rmse_garch}")

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(realized_vol /100 , label= "Realized Volatility")
plt.plot(np.sqrt(forecast_garch.variance.iloc[-len(split_date):]) /100 , label = "Volatility Prediction - GARCH")
plt.title("Volatility Prediction with GARCH", fontsize = 12)
plt.legend()
plt.show()

We can see that GARCH got a slight better value than ARCH, but still a bad value.

## Support Vector Regression with GARCH

In this section we complement the volatility estimates generated by the GARCH model with Support Vector Regression (SVR).. The goal is to allow a non-linear model to learn additional patterns that GARCH cannot capture. SVR is particularly suitable for financial volatility because it can model complex relationships while remaining robust to noise, a common characteristic of financial time series.

### Theoretical Background and Mechanism of SVR

Support Vector Regression (SVR) is an extension of Support Vector Machines (SVM) for regression tasks.  
SVR tries to find a function \(f(x) = w^T \phi(x) + b\) that approximates the target \(y\) with a maximum deviation of \(\epsilon\).  

It solves the following optimization problem:

$\min_{w,b} \frac{1}{2} \|w\|^2 + C \sum_{i=1}^{n} (\xi_i + \xi_i^*)$


subject to:

$$
\begin{cases} 
y_i - w^T \phi(x_i) - b \leq \epsilon + \xi_i \\
w^T \phi(x_i) + b - y_i \leq \epsilon + \xi_i^* \\
\xi_i, \xi_i^* \geq 0
\end{cases}
$$

Here, $\xi_i, \xi_i^*$ are slack variables for points outside the $\epsilon$-insensitive tube, $C$ is a regularization parameter controlling the trade-off between flatness and tolerance to deviations, and $\phi(x)$ is a (possibly nonlinear) feature mapping.

In essence, SVR minimizes both the model complexity $(\|w\|^2)$ and the error outside the $\epsilon$-tube, making it robust to noise.


### Why SVR is Useful with GARCH

GARCH models capture linear autoregressive patterns and volatility clustering but may miss nonlinear effects.  
SVR complements GARCH by learning residual volatility patterns not captured by the linear GARCH structure.  

Using kernels (linear, polynomial, RBF), SVR can model a variety of nonlinear relationships in volatility,
providing more flexible and robust predictions when combined with GARCH estimates.  

This hybrid approach leverages the strengths of both models: GARCH captures the main volatility dynamics,
and SVR refines the prediction by modeling residual nonlinearities.


### 

In [5]:
from sklearn.svm import SVR
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV

We use the realized volatility estimated from the rolling standard deviation of returns as the target variable for the SVR. This allows the SVR to learn from the actual volatility observed in the data, rather than relying solely on model-based estimates.

In [6]:
realized_vol = ret.rolling(5).std()
realized_vol = pd.DataFrame(realized_vol)
realized_vol.reset_index(drop=True, inplace=True)


NameError: name 'ret' is not defined

Input features are constructed using lagged squared returns. This captures volatility clustering and persistence, which are key characteristics of financial time series.

In [ ]:
returns_svm = ret ** 2
returns_svm = returns_svm.reset_index()
del returns_svm["Date"]

In [ ]:
X = pd.concat([realized_vol, returns_svm], axis=1, ignore_index=True)
X = X[4:].copy()
X = X.reset_index()
X.drop("index", axis=1, inplace=True)

In [ ]:
realized_vol = realized_vol.dropna().reset_index()
realized_vol.drop("index", axis=1, inplace=True)

We configure SVR models with multiple kernel functions: linear, polynomial, and radial basis function (RBF). Each kernel captures different types of relationships in the data. Linear kernel captures simple trends, polynomial kernel captures interactions between features, and RBF kernel captures more complex nonlinear patterns.

In [ ]:
svr_poly=SVR(kernel="poly", degree=2)
svr_lin=SVR(kernel="linear")
svr_rbf=SVR(kernel="rbf")

Hyperparameters are tuned using randomized cross-validation. This method efficiently searches through the hyperparameter space to find a balance between model complexity and predictive accuracy, reducing the risk of overfitting while improving out-of-sample performance.

In [ ]:
para_grid = {'gamma': sp_rand(), 'C': sp_rand(), 'epsilon': sp_rand()}
clf_lin = RandomizedSearchCV(svr_lin, para_grid, n_jobs=-1)
clf_lin.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))
predict_svr_lin = clf_lin.predict(X.iloc[-n:])

After training, the SVR model is used to predict out-of-sample volatility. Comparing these predictions with realized volatility allows us to assess the model's ability to capture real-world volatility fluctuations beyond what the GARCH model captures.

In [ ]:
predict_svr_lin = pd.DataFrame(predict_svr_lin)
predict_svr_lin.index = ret.iloc[-n:].index

We compute the root mean squared error (RMSE) to quantify the predictive performance of the SVR model. Lower RMSE indicates better alignment with realized volatility.

In [ ]:
rmse_svr = np.sqrt(mse(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100))
print(f"The SVR model with a linear kernel achieved an RMSE of {rmse_svr}")

Plotting the realized volatility against the SVR predictions provides a visual assessment of how well the model captures volatility patterns.

In [ ]:
realized_vol.index=ret.iloc[4:].index

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(realized_vol / 100, label="Realized Volatility")
plt.plot(predict_svr_lin / 100, label="SVR Prediction (Linear Kernel)")
plt.title("Volatility Prediction using SVR with Linear Kernel", fontsize=12)
plt.xlabel("Time")
plt.ylabel("Volatility")
plt.legend()
plt.show()

We repeat the same procedure for the SVR model with the RBF kernel. The RBF kernel is particularly effective for modeling complex nonlinear patterns, which are common in financial volatility.

In [ ]:
clf_rbf = RandomizedSearchCV(svr_rbf, para_grid, n_jobs=-1)
clf_rbf.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))
predict_svr_rbf = clf_rbf.predict(X.iloc[-n:])

In [ ]:
predict_svr_rbf = pd.DataFrame(predict_svr_rbf)
predict_svr_rbf.index = ret.iloc[-n:].index

In [ ]:
rmse_svr_rbf = np.sqrt(mse(realized_vol.iloc[-n:] / 100, predict_svr_rbf / 100))
print('The SVR model with an RBF kernel achieved an RMSE of {:.6f}'.format(rmse_svr_rbf))

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(realized_vol / 100, label="Realized Volatility")
plt.plot(predict_svr_rbf / 100, label="SVR Prediction (RBF Kernel)")
plt.title("Volatility Prediction using SVR with RBF Kernel", fontsize=12)
plt.xlabel("Time")
plt.ylabel("Volatility")
plt.legend()
plt.show()

### Results Discussion
The SVR model with a linear kernel achieves an RMSE of approximately 0.00143, while the SVR model with the RBF kernel has a higher RMSE of approximately 0.00302. This indicates that, for this dataset, the linear kernel provides more accurate predictions of realized volatility compared to the RBF kernel. The higher RMSE of the RBF kernel suggests that the additional flexibility of the nonlinear kernel may lead to overfitting or less effective generalization in this specific case. The plots confirm this, showing that the linear SVR predictions closely follow the realized volatility, whereas the RBF predictions deviate more from observed values.

## Neural Networks on GARCH

In [ ]:
from sklearn.neural_network import MLPRegressor
NN_vol = MLPRegressor(learning_rate_init=0.001, random_state=1)
para_grid_NN = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],'max_iter': [500, 1000],'alpha': [0.00005, 0.0005 ]}
clf_NN = RandomizedSearchCV(NN_vol, para_grid_NN)
clf_NN.fit(X.iloc[:-n].values,realized_vol.iloc[1:-(n-1)].values.reshape(-1, ))
NN_predictions = clf_NN.predict(X.iloc[-n:])

In [ ]:
NN_predictions= pd.DataFrame(NN_predictions)
NN_predictions.index=ret.iloc[-n:].index

In [ ]:
rmse_svr_nn = np.sqrt(mse(realized_vol.iloc[-n:] / 100, NN_predictions / 100))
print('The RMSE value of Neural Network is {:.6f}'.format(rmse_svr_rbf))

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(realized_vol / 100, label="Realized Volatility")
plt.plot(NN_predictions / 100, label="Volatility Prediction - SVR")
plt.title("Volatility Prediction with SVR", fontsize=12)
plt.legend()
plt.show()

## Deep Learning on GARCH

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

In [ ]:
model = keras.Sequential([layers.Dense(256, activation="relu"), layers.Dense(128, activation="relu"), layers.Dense(1, activation="linear")])

In [ ]:
model.compile(loss="mse", optimizer="rmsprop")

In [ ]:
epochs_trial= np.arange(100,400,4)
batch_trial= np.arange(100,400, 4)
DL_pred=[]
DL_RMSE=[]
for i, j, k in zip(range(4), epochs_trial, batch_trial):
    model.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,), batch_size=k, epochs=j, verbose=False)
    DL_predict = model.predict(np.asarray(X.iloc[-n:]))
    DL_RMSE.append(np.sqrt(mse(realized_vol.iloc[-n:] /100, DL_predict.flatten() /100)))
    DL_pred.append(DL_predict)
    print("DL_RMSE_{}:{:.6f}".format(i+1, DL_RMSE[i]))


In [ ]:
DL_predict= pd.DataFrame(DL_pred[DL_RMSE.index(min(DL_RMSE))])
DL_predict.index= ret.iloc[-n:].index

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(realized_vol / 100, label="Realized Volatility")
plt.plot(DL_predict / 100, label="Volatility Prediction - Deep Learning")
plt.title("Volatility Prediction with Deep Learning using GARCH", fontsize=12)
plt.legend()
plt.show()

As far as we tested, based on RMSE, a SVR using a linear kernel function was the best method to predict risk on PayPal stocks. This will be a useful feature to implement to predict stock direction.

# Predicting stock direction with LINEAR SVR GARCH  

In [ ]:
df_pypl['Predicted_Vol_SVR_Lin'] = predict_svr_lin
df_pypl_vol = df_pypl

In [ ]:
features = ["open", "high", "low", "close", "volume", "SMA_20", "SMA_50", "Predicted_Vol_SVR_Lin"]
X = df_pypl_vol [features]
y = df_pypl_vol["Target"]
print( "DataFrame da Paypal com features e Target")
print(df_pypl_vol[features + ["Target"]].tail())
print(f"\nNúmero de amostras prontas para o SVM (X e y): {X.shape[0]}")
print(f"Número de Features (X): {X.shape[1]}")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=False)
np.random.seed(42)
num_samples= X.shape[0]


In [ ]:
scaler = StandardScaler()
X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

